# WCRP Coupled Model Intercomparison Project (CMIP)

CMIP -> Calcular la sensibilidad de clima mediante un ensamble de modelos

The uncertainty associated with the speed of temperature rise is associated with the limitations of our understanding of the global climate system. Each general circulation model used by the IPCC and included in the CMIP5 ensemble uses different assumptions and parameter values to describe the atmospheric changes resulting in growing anthropogenic GHG emissions, and, as a result, the magnitude of the estimated changes varies greatly among different modeling groups. In this respect, one of the features of EDIAM is that it uses 12 GCMs included in the CMIP5 data ensemble to calibrate the parameters


* $\beta$ :  sensibilidad de la atmósfera a emisiones de $CO_2$ (grados celsius)
* $\xi$ : la capacidad de sumidero de carbono de la atmósfera (ppm/BTU/year)
* $\delta$: tasa promedio de regeneración ambiental natural

In [1]:
## Cargamos bibliotecas
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import math
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import scipy.stats
import math
import os

In [2]:
#PART1: CALIBRATION OF DELTA TEMP VS LOG CO2 FUNCTION
# Cargamos los datos del csv AllGCMs.csv, los cuales contienen
climate_models = pd.read_csv("/home/milo/PCIC/Maestría/2doSemestre/seminario/github/data/cmip6/gcm_year_cmip6.csv")
co2_df = pd.read_csv("/home/milo/PCIC/Maestría/2doSemestre/seminario/github/data/cmip6/co2_gcm_year_cmip6.csv")

climate_models = pd.concat([climate_models,co2_df])

climate_models = climate_models.query("scenario!='historical'")
#climate_models = climate_models.query("year < 2096")
# [degrees Celsius] increase in average global temperature
delta_temp_disater = 8  

cmip_models = ["CESM2-WACCM","GFDL-ESM4"]

def linear_reg(datos,modelo,region,intercepto):
    print("Climate model:{}\nRegion:{}".format(modelo,region))
    model_data = datos.query("climate_model=='{}'".format(modelo))
    #if modelo == "MRI-ESM1":
    #    model_data = model_data.query("year <2091")
    model_data = model_data.query("region=='{}'".format(region)) 
    n = model_data.query("variable == 'co2'").shape[0]

    X = model_data.query("variable == 'co2'")["value"].apply(math.log).values.reshape(n,1)
    Y = model_data.query("variable == 'tas'")["value"].values.reshape(n,1)

    if intercepto:
        regress = LinearRegression().fit(X,Y)
        R2 = regress.score(X,Y)
        Y_pred= regress.predict(X)
        MSE = mean_squared_error(Y_pred,Y)
        b0 = regress.intercept_[0]
        b1 = regress.coef_[0][0]

        var_b0= (1/n) * ((MSE*np.square(X).sum())/np.square(X -X.mean()).sum())
        var_b1 = MSE/np.square(X -X.mean()).sum()
        t_b0 = b0/math.sqrt(var_b0)
        t_b1 = b1/math.sqrt(var_b1)

        pvalue_b0 = scipy.stats.t.sf(abs(t_b0),n-2)
        pvalue_b1 = scipy.stats.t.sf(abs(t_b1),n-2)

        d = {"climate_model": [modelo],
             "region" : [region],
             "beta_delta_temp":[b1],
             "intercept":[b0],
             "adjR_squared": [R2],
             "intercept_P_value": [pvalue_b0],
             "beta_P_value":  [pvalue_b0]}
        df = pd.DataFrame(data=d)
    else:
        regress = LinearRegression(fit_intercept=False).fit(X,Y)
        df = regress.coef_[0][0]
        

    return df

# Para cada modelo realizamos una regresión lineal entre log(co2.ppm) y tas.anomaly
result = pd.DataFrame()

for model in cmip_models:
    for region in climate_models["region"].unique():
        result = result.append(linear_reg(climate_models,model,region,True), ignore_index=True)

#estimate CO2 concentrations for delta temp disaster

result['co2_base'] = (-1*result['intercept']/result['beta_delta_temp']).apply(math.exp)
result['co2_disaster'] =result['co2_base']*(delta_temp_disater/result['beta_delta_temp']).apply(math.exp)
result['delta_temp_disaster'] = delta_temp_disater
result

Climate model:CESM2-WACCM
Region:america
Climate model:CESM2-WACCM
Region:eurafrica
Climate model:CESM2-WACCM
Region:asia
Climate model:GFDL-ESM4
Region:america
Climate model:GFDL-ESM4
Region:eurafrica
Climate model:GFDL-ESM4
Region:asia


,climate_model,region,beta_delta_temp,intercept,adjR_squared,intercept_P_value,beta_P_value,co2_base,co2_disaster,delta_temp_disaster
0,CESM2-WACCM,america,5.238091,-31.116062,0.954571,0.0,0.0,380.065674,1750.432339,8
1,CESM2-WACCM,eurafrica,5.299541,-31.480907,0.959169,0.0,0.0,380.052181,1719.645298,8
2,CESM2-WACCM,asia,5.391266,-32.025688,0.959310,0.0,0.0,380.045394,1676.011581,8
3,GFDL-ESM4,america,3.389631,-20.099045,0.933021,0.0,0.0,375.991738,3982.663130,8
4,GFDL-ESM4,eurafrica,3.643759,-21.605849,0.926541,0.0,0.0,375.985117,3378.153355,8
5,GFDL-ESM4,asia,3.504845,-20.782377,0.934067,0.0,0.0,376.009540,3685.529032,8


In [3]:
climate_models = climate_models[["year","climate_model","region","variable","value"]].groupby(["year","climate_model","region","variable"], as_index=False).mean()
climate_models = climate_models.pivot(index= ["year","climate_model","region"], columns ="variable",values="value").reset_index()
climate_models

variable,year,climate_model,region,co2,co2mass,tas
0,1850,CESM2-WACCM,america,286.337367,2.223374e+15,-1.649295
1,1850,CESM2-WACCM,asia,286.333088,2.223374e+15,-1.268030
2,1850,CESM2-WACCM,eurafrica,286.332186,2.223374e+15,-1.191420
3,1850,GFDL-ESM4,america,284.376816,2.219970e+15,-0.855544
4,1850,GFDL-ESM4,asia,284.390931,2.219970e+15,-1.041876
...,...,...,...,...,...,...
2254,2100,GFDL-ESM4,asia,749.405328,5.842545e+15,2.351517
2255,2100,GFDL-ESM4,eurafrica,749.342740,5.842545e+15,2.449535
2256,2100,NorESM2-LM,america,282.592096,3.900457e+15,2.187378
2257,2100,NorESM2-LM,asia,282.592089,3.900457e+15,2.293307


In [4]:
result["id"] =result["climate_model"] + result["region"]
climate_models["id"] =climate_models["climate_model"] + climate_models["region"]


In [5]:
#PART2: CALIBRATION OF S EQUATION
#climate_models = climate_models.query("year < 2091")
#merge CO2 Disaster with raw data
climate_models = climate_models.merge(result[['id','co2_disaster']], how='inner', on='id')
climate_models.drop(columns="id",inplace=True)
climate_models

,year,climate_model,region,co2,co2mass,tas,co2_disaster
0,1850,CESM2-WACCM,america,286.337367,2.223374e+15,-1.649295,1750.432339
1,1851,CESM2-WACCM,america,286.388356,2.224432e+15,-1.841042,1750.432339
2,1852,CESM2-WACCM,america,286.456983,2.225580e+15,-1.494777,1750.432339
3,1853,CESM2-WACCM,america,286.578987,2.226614e+15,-1.555146,1750.432339
4,1854,CESM2-WACCM,america,286.696682,2.227506e+15,-1.671045,1750.432339
...,...,...,...,...,...,...,...
1501,2096,GFDL-ESM4,eurafrica,728.202205,5.677010e+15,2.377425,3378.153355
1502,2097,GFDL-ESM4,eurafrica,733.317160,5.716184e+15,2.321409,3378.153355
1503,2098,GFDL-ESM4,eurafrica,738.835239,5.761206e+15,2.412418,3378.153355
1504,2099,GFDL-ESM4,eurafrica,744.296085,5.804469e+15,2.419729,3378.153355


In [6]:
#calculate quality of the environment
climate_models['s'] = climate_models['co2_disaster'] - climate_models['co2']

#order data set
climate_models = climate_models.sort_values(by = ['climate_model','year'])
climate_models

,year,climate_model,region,co2,co2mass,tas,co2_disaster,s
0,1850,CESM2-WACCM,america,286.337367,2.223374e+15,-1.649295,1750.432339,1464.094973
251,1850,CESM2-WACCM,asia,286.333088,2.223374e+15,-1.268030,1676.011581,1389.678493
502,1850,CESM2-WACCM,eurafrica,286.332186,2.223374e+15,-1.191420,1719.645298,1433.313112
1,1851,CESM2-WACCM,america,286.388356,2.224432e+15,-1.841042,1750.432339,1464.043983
252,1851,CESM2-WACCM,asia,286.377704,2.224432e+15,-1.370785,1676.011581,1389.633877
...,...,...,...,...,...,...,...,...
1253,2099,GFDL-ESM4,asia,744.302648,5.804469e+15,2.399624,3685.529032,2941.226384
1504,2099,GFDL-ESM4,eurafrica,744.296085,5.804469e+15,2.419729,3378.153355,2633.857270
1003,2100,GFDL-ESM4,america,749.312880,5.842545e+15,2.416860,3982.663130,3233.350251
1254,2100,GFDL-ESM4,asia,749.405328,5.842545e+15,2.351517,3685.529032,2936.123704


In [7]:
#load data of world consumption of fossil fuels
fossil_fuel = pd.read_csv("/home/milo/PCIC/Maestría/2doSemestre/seminario/github/data/climate_data_calibration/energia_fosil_total_regiones.csv")
fossil_fuel = fossil_fuel.query("energia =='fosil'")
fossil_fuel.loc[fossil_fuel["region"]=="America","region"] = "america"
fossil_fuel.loc[fossil_fuel["region"]=="Asia-Oceania","region"] = "asia"
fossil_fuel.loc[fossil_fuel["region"]=="Eurafrica","region"] = "eurafrica"

fossil_fuel = fossil_fuel[["year","region","value"]].rename(columns = {'value':'fossil_fuels_consumption'})
fossil_fuel

,year,region,fossil_fuels_consumption
0,1980,america,89.46194
1,1980,asia,52.11488
2,1980,eurafrica,113.72357
3,1981,america,86.67770
4,1981,asia,52.50440
...,...,...,...
94,2011,asia,222.85927
95,2011,eurafrica,108.06058
96,2012,america,114.84215
97,2012,asia,227.25261


In [8]:
def compute_delta(datos,modelo,region):
    datos = datos.query("climate_model == '{}' and region == '{}'".format(modelo,region)).reset_index(drop=True)
    datos["s_diff"]= datos["s"].diff() 
    model_data = datos.merge(fossil_fuel.query("region == '{}'".format(region))[["year","fossil_fuels_consumption"]], how = "right" ,on = "year" )    

        
    n = model_data.shape[0]

    X = model_data['fossil_fuels_consumption'].values.reshape(n,1)
    Y = model_data['s_diff'].values.reshape(n,1)

    regress = LinearRegression(fit_intercept=False).fit(X,Y)
    qsi =   regress.coef_[0][0]/-0.5

    #estimate Delta.S

    model_data['s_lag'] = model_data['s']-model_data['s_diff']
    Y = model_data['s_lag'].values.reshape(n,1)
    regress = LinearRegression(fit_intercept=False).fit(X,Y)
    Delta_S = (0.5*qsi)/regress.coef_[0][0]

    #calculate S_hat
    model_data['s_hat'] = (-1*qsi*model_data['fossil_fuels_consumption'])+((1+Delta_S)*model_data['s_lag'])
    
    d = {"climate_model" :[modelo],
         "region" : [region],
         "qsi":[qsi],
         "delta_s":[Delta_S],
         "s_0":model_data[model_data["year"]==2012]["s_hat"].values}
    df = pd.DataFrame(data=d)
    return df

In [9]:
#we make the same assumption as Acemoglous et. al, such that Delta.S=0.5(qsi*Y(t)/S(t-1));
#thus the original model: S(t)=-qsi*Y(t)+(1+Delta.S)*S(t-1) is reduced to: S(t)-S(t-1)=-0.5*qsi*Y(t)
#and can be estimated with lm

s_parameters = pd.DataFrame()

for model in climate_models["climate_model"].unique():
    for region in ["america","eurafrica","asia"]:
        s_parameters = s_parameters.append(compute_delta(climate_models,model,region), ignore_index=True)

s_parameters


,climate_model,region,qsi,delta_s,s_0
0,CESM2-WACCM,america,0.032838,0.001237,1357.621507
1,CESM2-WACCM,eurafrica,0.030342,0.001245,1327.309065
2,CESM2-WACCM,asia,0.025260,0.001361,1281.320409
3,GFDL-ESM4,america,0.032224,0.000464,3595.476205
4,GFDL-ESM4,eurafrica,0.029823,0.000550,2991.417501
5,GFDL-ESM4,asia,0.024713,0.000524,3296.437825


In [10]:
#Create a table with parameters for all climate models
climate_param_ini = result[["climate_model","beta_delta_temp","co2_base","co2_disaster","delta_temp_disaster","id"]]
s_parameters["id"] = s_parameters["climate_model"] + s_parameters["region"]
s_parameters.drop(columns="climate_model",inplace=True)
climate_param = climate_param_ini.merge(s_parameters,how ="inner", on="id")
climate_param.drop(columns='id',inplace=True)
climate_param

,climate_model,beta_delta_temp,co2_base,co2_disaster,delta_temp_disaster,region,qsi,delta_s,s_0
0,CESM2-WACCM,5.238091,380.065674,1750.432339,8,america,0.032838,0.001237,1357.621507
1,CESM2-WACCM,5.299541,380.052181,1719.645298,8,eurafrica,0.030342,0.001245,1327.309065
2,CESM2-WACCM,5.391266,380.045394,1676.011581,8,asia,0.025260,0.001361,1281.320409
3,GFDL-ESM4,3.389631,375.991738,3982.663130,8,america,0.032224,0.000464,3595.476205
4,GFDL-ESM4,3.643759,375.985117,3378.153355,8,eurafrica,0.029823,0.000550,2991.417501
5,GFDL-ESM4,3.504845,376.009540,3685.529032,8,asia,0.024713,0.000524,3296.437825


In [11]:
climate_param.to_csv("/home/milo/PCIC/Maestría/2doSemestre/seminario/github/data/experimental_design/cmip6_climate_param.csv",index=False)

In [12]:
climate_param.to_dict('index')

{0: {'climate_model': 'CESM2-WACCM',
  'beta_delta_temp': 5.238090792566205,
  'co2_base': 380.0656739299596,
  'co2_disaster': 1750.4323392001681,
  'delta_temp_disaster': 8,
  'region': 'america',
  'qsi': 0.03283822564851918,
  'delta_s': 0.001236650160936777,
  's_0': 1357.6215072101104},
 1: {'climate_model': 'CESM2-WACCM',
  'beta_delta_temp': 5.2995407563394075,
  'co2_base': 380.0521806781624,
  'co2_disaster': 1719.645298153347,
  'delta_temp_disaster': 8,
  'region': 'eurafrica',
  'qsi': 0.0303418344260333,
  'delta_s': 0.0012449079258612238,
  's_0': 1327.309064516354},
 2: {'climate_model': 'CESM2-WACCM',
  'beta_delta_temp': 5.391266122361606,
  'co2_base': 380.045393585297,
  'co2_disaster': 1676.011581395193,
  'delta_temp_disaster': 8,
  'region': 'asia',
  'qsi': 0.025260423094237056,
  'delta_s': 0.0013605866198733688,
  's_0': 1281.3204093819343},
 3: {'climate_model': 'GFDL-ESM4',
  'beta_delta_temp': 3.389631105911461,
  'co2_base': 375.9917382950945,
  'co2_disas

### ENERGIAS POR ECONOMÍAS

In [13]:
fosil_regiones_lideres = pd.read_csv("/home/milo/PCIC/Maestría/2doSemestre/seminario/github/data/climate_data_calibration/energia_fosil_desagregado_lideres_regiones.csv")
renovables_regiones_lideres = pd.read_csv("/home/milo/PCIC/Maestría/2doSemestre/seminario/github/data/climate_data_calibration/energia_renovable_total_lideres_regiones.csv")

list(fosil_regiones_lideres.query("economia=='advanced' and (energia=='carbon' or energia =='petroleo')").sort_values(by="year").value)

[22.15037,
 14.54074,
 40.88987,
 20.55854,
 38.25728,
 1.54837,
 21.61148,
 38.42535000000001,
 21.059220000000003,
 35.55293,
 13.76381,
 1.50822,
 38.10741,
 12.96325,
 33.35381,
 22.33587,
 20.25839,
 1.4411999999999998,
 37.23772,
 23.72908,
 32.9227,
 1.56643,
 12.71399,
 19.12118,
 34.100719999999995,
 1.94094,
 36.70682,
 20.44938,
 13.31105,
 26.41226,
 28.46968,
 19.90765,
 36.73697,
 2.06259,
 13.08079,
 33.96105,
 18.75167,
 35.247249999999994,
 37.59639,
 29.81907,
 13.41643,
 2.1213,
 19.78348,
 38.00791,
 36.04638,
 31.14549,
 13.78948,
 2.18446,
 37.55738,
 14.743190000000002,
 2.27334,
 32.65432,
 20.81902,
 37.97109,
 15.3383,
 33.56461,
 37.69222,
 37.69718,
 2.39189,
 22.06255,
 2.7107,
 22.01408,
 15.67067,
 36.98597,
 34.005720000000004,
 36.83358,
 16.22829,
 36.13862,
 36.32576,
 2.88555,
 22.46945,
 42.41992,
 36.90892,
 16.73035,
 27.50533,
 2.89379,
 34.83236000000001,
 23.35345,
 31.454589999999996,
 2.96397,
 37.16007,
 15.69595,
 23.96536,
 27.36988,
 18.0

In [14]:
renovables_regiones_lideres.query("economia=='emerging' and region =='Eurafrica'").sort_values(by="year")

,year,region,energia,value,economia
165,1980,Eurafrica,renovable,2.17530,emerging
166,1981,Eurafrica,renovable,2.24729,emerging
167,1982,Eurafrica,renovable,2.24942,emerging
168,1983,Eurafrica,renovable,2.35983,emerging
169,1984,Eurafrica,renovable,2.23377,emerging
170,1985,Eurafrica,renovable,2.22566,emerging
171,1986,Eurafrica,renovable,2.22165,emerging
172,1987,Eurafrica,renovable,2.33660,emerging
173,1988,Eurafrica,renovable,2.41370,emerging
174,1989,Eurafrica,renovable,2.47249,emerging


In [15]:
def get_initial_value_energy(economia,tipo_energia,region):
    if tipo_energia =='fosil':
        consulta = "economia=='{}' and (energia=='carbon' or energia =='petroleo') and region=='{}' and year ==2012".format(economia,region)     
        return sum(fosil_regiones_lideres.query(consulta)["value"])
    
    elif tipo_energia =='renovable':
        consulta = "economia=='{}' and energia=='renovable' and region=='{}' and year ==2012".format(economia,region)
        return sum(renovables_regiones_lideres.query(consulta)["value"])
    
    

In [16]:
Yre_N_0 = []
Yce_N_0 = []
Yre_S_0 = []
Yce_S_0 = []


regiones = ['America', 'Eurafrica', 'Asia-Oceania']
economias = ["advanced","emerging"]
energias = ["fosil","renovable"]

for en in energias:
    for eco in economias:
        for region in regiones:
            if en == "fosil":
                if eco == "advanced":
                    Yce_N_0.append(get_initial_value_energy(eco,en,region))
                elif eco == "emerging":
                    Yce_S_0.append(get_initial_value_energy(eco,en,region))
            elif en == "renovable":
                if eco == "advanced":
                    Yre_N_0.append(get_initial_value_energy(eco,en,region))
                elif eco == "emerging":
                    Yre_S_0.append(get_initial_value_energy(eco,en,region)) 

In [17]:
climate_param["Yre_N_0"] = Yre_N_0*2
climate_param["Yce_N_0"] = Yce_N_0*2
climate_param["Yre_S_0"] = Yre_S_0*2
climate_param["Yce_S_0"] = Yce_S_0*2
climate_param.to_csv("/home/milo/PCIC/Maestría/2doSemestre/seminario/github/data/experimental_design/cmip6_climate_param.csv",index=False)

In [18]:
climate_param

,climate_model,beta_delta_temp,co2_base,co2_disaster,delta_temp_disaster,region,qsi,delta_s,s_0,Yre_N_0,Yce_N_0,Yre_S_0,Yce_S_0
0,CESM2-WACCM,5.238091,380.065674,1750.432339,8,america,0.032838,0.001237,1357.621507,8.82351,68.43374,7.79899,26.59169
1,CESM2-WACCM,5.299541,380.052181,1719.645298,8,eurafrica,0.030342,0.001245,1327.309065,8.46526,61.65342,3.71644,23.22470
2,CESM2-WACCM,5.391266,380.045394,1676.011581,8,asia,0.025260,0.001361,1281.320409,10.71390,39.95640,5.54137,83.70293
3,GFDL-ESM4,3.389631,375.991738,3982.663130,8,america,0.032224,0.000464,3595.476205,8.82351,68.43374,7.79899,26.59169
4,GFDL-ESM4,3.643759,375.985117,3378.153355,8,eurafrica,0.029823,0.000550,2991.417501,8.46526,61.65342,3.71644,23.22470
5,GFDL-ESM4,3.504845,376.009540,3685.529032,8,asia,0.024713,0.000524,3296.437825,10.71390,39.95640,5.54137,83.70293
